In [ ]:
import os
import sys
os.chdir("/home/lyt/checker_finallap") # your working path
sys.path.append("/home/lyt/checker_finallap")

import BRICK
url = "neo4j://10.224.28.66:7687"
auth = ("neo4j", "bmVvNGpwYXNzd29yZA==")

BRICK.config(url=url, auth=auth)

import scanpy as sc
import anndata
from BRICK import pp, qr, rk
import pandas as pd
import numpy as np

# Define the prune_parent_cell_types function
def prune_parent_cell_types(candidate_cell_types):
    # Query is_a relationships between cell types
    qg = BRICK.qr.query_relation(candidate_cell_types)
    
    # Filter and process the query graph to extract normalized path information
    paths = filter_results(qg)
    
    # Identify parent cell types with only one subtype
    parent_to_subtypes = {}
    for source, relation, target in paths:
        if relation == "is_a":
            if source not in parent_to_subtypes:
                parent_to_subtypes[source] = []
            parent_to_subtypes[source].append(target)
    
    # Replace parent cell types with their subtypes if they have only one subtype
    pruned_cell_types = []
    for cell_type in candidate_cell_types:
        if cell_type in parent_to_subtypes and len(parent_to_subtypes[cell_type]) == 1:
            pruned_cell_types.append(parent_to_subtypes[cell_type][0])
        else:
            pruned_cell_types.append(cell_type)
    
    # Create a pruned query result dataframe
    pruned_df = pd.DataFrame({"cell_type": pruned_cell_types})
    
    return pruned_df

# Load the data
adata = anndata.read_h5ad('/home/lyt/checker_finallap/files/adata_new1.h5ad')

# Perform Leiden clustering analysis
sc.tl.leiden(adata, key_added='leiden', flavor='igraph', n_iterations=2, directed=False)

# Calculate differentially expressed genes for each cluster relative to other clusters
with np.errstate(invalid='ignore'):
    sc.tl.rank_genes_groups(adata, groupby='leiden', pts=True)

# Convert differential expression results format using BRICK function
pp.rank_genes_groups2df(adata)

/home/lyt/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Graph database has been configured and initialized successfully.
